# 先物データ探索

このノートブックでは、Azure SQL Databaseに接続してLME、SHFE、CMXの銅先物データを取得・表示します。

## 1. 必要なライブラリのインポート

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# プロジェクトのルートディレクトリをPythonパスに追加
project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, project_root)

# 設定のインポート
from config.database_config import get_connection_string

# 警告を抑制
warnings.filterwarnings('ignore')

# プロット設定
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("ライブラリのインポートが完了しました。")

## 2. データベース接続関数の定義

In [ ]:
def connect_to_database():
    """Azure SQL Databaseに接続"""
    try:
        conn_string = get_connection_string()
        conn = pyodbc.connect(conn_string)
        print("データベースに正常に接続しました。")
        return conn
    except Exception as e:
        print(f"データベース接続エラー: {e}")
        return None

def execute_query(conn, query):
    """SQLクエリを実行してDataFrameを返す"""
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy")
            df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        print(f"クエリ実行エラー: {e}")
        return None

## 3. データベース接続テスト

In [ ]:
# データベースに接続
conn = connect_to_database()

if conn:
    # マスタデータの確認
    query = """
    SELECT 
        MetalCode,
        MetalName,
        CurrencyCode,
        ExchangeCode
    FROM M_Metal
    ORDER BY MetalID
    """
    
    metals_df = execute_query(conn, query)
    if metals_df is not None:
        print("\n登録されている金属マスタ:")
        display(metals_df)

## 4. 限月タイプの確認

In [ ]:
if conn:
    query = """
    SELECT 
        TenorTypeID,
        TenorTypeName,
        Description
    FROM M_TenorType
    WHERE TenorTypeName LIKE '%Future%'
    ORDER BY TenorTypeID
    """
    
    tenor_types_df = execute_query(conn, query)
    if tenor_types_df is not None:
        print("\n先物関連の限月タイプ:")
        display(tenor_types_df)

## 5. LME銅先物データの取得

In [ ]:
if conn:
    # 最新30日間のLME銅先物データを取得
    query = """
    SELECT 
        p.TradeDate,
        m.MetalCode,
        m.ExchangeCode,
        t.TenorTypeName,
        p.SettlementPrice,
        p.OpenPrice,
        p.HighPrice,
        p.LowPrice,
        p.LastPrice,
        p.Volume,
        p.OpenInterest
    FROM T_CommodityPrice p
    INNER JOIN M_Metal m ON p.MetalID = m.MetalID
    INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
    WHERE 
        m.MetalCode = 'COPPER'
        AND t.TenorTypeName LIKE 'Generic%Future%'
        AND p.TradeDate >= DATEADD(day, -30, GETDATE())
    ORDER BY p.TradeDate DESC, t.TenorTypeID
    """
    
    lme_futures_df = execute_query(conn, query)
    if lme_futures_df is not None:
        print(f"\nLME銅先物データ: {len(lme_futures_df)}件取得")
        print("\n最新データ（上位10件）:")
        display(lme_futures_df.head(10))

## 6. データサマリーの表示

In [ ]:
if conn and lme_futures_df is not None and not lme_futures_df.empty:
    # 日付範囲
    print(f"\nデータ期間: {lme_futures_df['TradeDate'].min()} から {lme_futures_df['TradeDate'].max()}")
    
    # 限月別の統計
    summary_stats = lme_futures_df.groupby('TenorTypeName')['SettlementPrice'].agg([
        'count', 'mean', 'std', 'min', 'max'
    ]).round(2)
    
    print("\n限月別の価格統計:")
    display(summary_stats)

## 7. 最新取引日の先物カーブ

In [ ]:
if conn and lme_futures_df is not None and not lme_futures_df.empty:
    # 最新取引日のデータを抽出
    latest_date = lme_futures_df['TradeDate'].max()
    latest_data = lme_futures_df[lme_futures_df['TradeDate'] == latest_date].copy()
    
    # 限月番号を抽出して順序を設定
    latest_data['TenorNumber'] = latest_data['TenorTypeName'].str.extract(r'Generic (\d+)')[0].astype(int)
    latest_data = latest_data.sort_values('TenorNumber')
    
    print(f"\n最新取引日 ({latest_date}) の先物価格:")
    display(latest_data[['TenorTypeName', 'SettlementPrice', 'Volume', 'OpenInterest']])

## 8. データベース接続のクローズ

In [ ]:
if conn:
    conn.close()
    print("データベース接続をクローズしました。")